<a href="https://colab.research.google.com/github/kumagaimasahito/Qsort/blob/main/quantum-sort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What Qsort Do

In [1]:
# Qsortをインストール
!pip install git+https://github.com/kumagaimasahito/Qsort.git

  Cloning https://github.com/kumagaimasahito/Qsort.git to /tmp/pip-req-build-vl7bcgof
  Running command git clone -q https://github.com/kumagaimasahito/Qsort.git /tmp/pip-req-build-vl7bcgof
     |████████████████████████████████| 92kB 3.8MB/s 
     |████████████████████████████████| 399kB 18.1MB/s 
     |████████████████████████████████| 102kB 8.1MB/s 
     |████████████████████████████████| 102kB 9.5MB/s 
     |████████████████████████████████| 409kB 16.8MB/s 
     |████████████████████████████████| 3.3MB 26.1MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 6.1MB 15.9MB/s 
     |████████████████████████████████| 215kB 44.9MB/s 
     |████████████████████████████████| 798kB 35.9MB/s 
     |████████████████████████████████| 32.7MB 129kB/s 
  Created wheel for Qsort: filename=Qsort-0.1.0-cp36-none-any.whl size=3619 sha256=469044a88e86f0d739a9f324b0246dbbc3e15d48ac7d8945f4a07773019fc886
  Stored in directory: /tmp/pip-ephem-wheel-cache-dg

In [2]:
# 問題設定
import random
num_numbers = 4
min_number = 1
max_number = 100
numbers = [random.randint(min_number, max_number) for _ in range(num_numbers)]

print("index : numbers")
for i, x in enumerate(numbers):
    print(i, "      : ", x)

index : numbers
0       :  43
1       :  51
2       :  19
3       :  7


In [3]:
# 差分行列を生成
diffs = {
    (v, w) : abs(numbers[w] - numbers[v]) 
    for v in range(num_numbers) 
    for w in range(v+1, num_numbers)
}

print("pair         : difference")
for pair, diff in diffs.items():
    print(pair, "      : ", diff)

pair         : difference
(0, 1)       :  8
(0, 2)       :  24
(0, 3)       :  36
(1, 2)       :  32
(1, 3)       :  44
(2, 3)       :  12


In [4]:
# 隣接行列を生成
neighbors = [
    (i, j)
    for i in range(num_numbers) 
    for j in range(num_numbers)
    if abs(j-i) == 1
]

print("neighbor")
for pair in neighbors:
    print(pair)

neighbor
(0, 1)
(1, 0)
(1, 2)
(2, 1)
(2, 3)
(3, 2)


In [5]:
# ラグランジュ未定乗数の設定
max_number = max(numbers)
lam = max_number/2

print(max_number)
print(lam)

51
25.5


In [6]:
# QUBOを生成
qubo = {
    ((v, i), (w, j)) : 
        diffs[(v, w)] if (i, j) in neighbors and (v, w) in diffs.keys() else
        -2 * lam if v == w and i == j else
        2 * lam if v == w and i < j else
        2 * lam if i == j else
        0
    for v in range(num_numbers)
    for w in range(v, num_numbers)
    for i in range(num_numbers)
    for j in range(num_numbers)
}

qubo = {k : v for k, v in qubo.items() if not v == 0}

In [7]:
# ひとまず，SA（シミュレーティド・アニーリング）で試してみる
import neal
sampler = neal.SimulatedAnnealingSampler()

In [8]:
# QUBOを渡して，SAを10回実行
response = sampler.sample_qubo(qubo, num_reads=10)

In [9]:
# 得られた回のうち，エネルギー最小のものをsolutionsとして取得
solutions = [state.tolist() for i, state in enumerate(response.record['sample']) if response.record['energy'][i] == min(response.record['energy'])]

for sol in solutions:
    print(sol)

[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
[0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
[0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]


In [10]:
selected_sols = []
for sol in solutions:
    if not sol in selected_sols:
        selected_sols.append(sol)

for sol in selected_sols:
    print(sol)

[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
[0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0]


# How to Use Qsort

In [11]:
from Qsort import Qsort
qsort = Qsort()
qsort.set_random_numbers()

index : numbers
0       :  59
1       :  71
2       :  13
3       :  70


In [12]:
sorted_numbers = qsort.SASolver()
print(sorted_numbers)

[71, 70, 59, 13]


In [14]:
sorted_numbers = qsort.QASolver(
    token = "Your API Token",
    solver = "Advantage_system1.1",
    endpoint = "https://cloud.dwavesys.com/sapi/"
)
print(sorted_numbers)

[71, 70, 59, 13]
